# Effects of not applying tabu moves

In [1]:
import random

import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

import tengp
from gpbenchmarks import get_data

def pdivide(x, y):
    return np.divide(x, y, out=np.copy(x), where=x!=0)

def plog(x):
    return np.log(x, out=np.copy(x), where=x>0)



funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.subtract, 2)
funset.add(np.multiply, 2)
funset.add(pdivide, 2)
funset.add(plog, 1)
funset.add(np.sin, 1)
funset.add(np.cos, 1)

params1d = tengp.Parameters(2, 1, 1, 100, funset)
params2d = tengp.Parameters(3, 1, 1, 100, funset)

#functions = [f'nguyenf{i}' for i in range(1,9)]
functions = [('nguyenf4', params1d, [20, -1, 1]), ('nguyenf7', params1d, [20, 0, 2]), ('nguyenf10', params2d, [100, -1, 1])]

TRIALS = 50

In [2]:
MUTATIONS = tengp.mutations.MUTATIONS

@tengp.utils.handle_invalid_decorator
def tabu_es(X, y, cost_function, params,
            target_fitness=0,
            population_size=5,
            evaluations=5000,
            random_state=None,
            mutation='point',
            mutation_probability=0.25,
            memory_size=10,
            verbose=False):

    if mutation not in MUTATIONS:
        raise UnknownMutationException("Provided type of mutation is not implemented.")

    move = MUTATIONS[mutation]
    if mutation == 'probabilistic':
        move = partial(move, probability=mutation_probability)

    if random_state:
        random.seed(random_state)

    # initialize memory
    memory = []
    memory_hits = 0

    # initial generation
    ib = tengp.individual.IndividualBuilder(params)

    population = [ib.create() for _ in range(population_size)]

    n_evals = 0
    
    stats = {
        'memory_hits': 0,
        'better_after': 0,
        'worse_after': 0
    }

    generation = 0

    for individual in population:
        output = individual.transform(X)
        individual.fitness = cost_function(y, output)
        n_evals += 1

    while n_evals < evaluations:
        generation += 1

        parent = min(population, key=lambda x: x.fitness)

        if parent.fitness <= target_fitness:
            return population

        population = []


        for _ in range(population_size -1):
            _move = move(parent)

            while _move in memory:
                stats['memory_hits'] += 1
                _move = move(parent)
                output = parent.apply(_move).transform(X)
                fitness = cost_function(y, output)
                if fitness < parent.fitness:
                    stats['better_after'] += 1
                elif fitness > parent.fitness:
                    stats['worse_after'] += 1
                

            individual = parent.apply(_move)

            if individual == parent:
                continue

            if len(memory) > memory_size:
                del memory[0]

            memory.append(_move)

            population.append(parent.apply(_move))


        for individual in population:
            if individual == parent:
                individual.fitness = parent.fitness
                continue
            output = individual.transform(X)
            individual.fitness = cost_function(y, output)
            n_evals += 1

        population += [parent]

        if verbose and generation % 100 == 0:
            print(f'Gen: {generation}, population: {sorted([x.fitness for x in population])}')

    population.sort(key=lambda x: x.fitness)
    return population, stats


In [11]:
%%time

random.seed(0)

ts_results = []

for function in functions:
    trial_results = []
    for trial in range(TRIALS):
        print(trial, end=',')
        X, y = get_data(function[0], *function[2])
        X = np.c_[np.ones(len(X)), X]
        population, stats = tabu_es(X, y, mean_squared_error, function[1], mutation='single', memory_size=1000)
        trial_results.append((population[0].fitness, stats['memory_hits'], stats['better_after'], stats['worse_after']))
    ts_results.append(trial_results)
    

0,

/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in multiply
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in sin
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in cos
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in greater
  from ipykernel import kernelapp as app


1,2,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in add
  current_node.value = current_node.fun(*values)


3,4,5,

/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in subtract
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,0,1,2,3,4,5,6,7,8,9,10,11,

ValueError: too many values to unpack (expected 2)

In [10]:
ts_results

[[(0.0056868445327356975, 103, 0, 99),
  (0.004448076656628229, 177, 1, 166),
  (0.04096606444930617, 113, 0, 105),
  (0.023387360592215283, 179, 1, 152),
  (0.011116881371513409, 208, 0, 171),
  (0.011915601050722901, 188, 0, 169),
  (0.03420129797931197, 150, 1, 132),
  (0.005871946376804391, 93, 0, 93),
  (0.0061395392318011255, 161, 0, 138),
  (0.19658755854075075, 242, 0, 216),
  (0.3315787877962767, 188, 0, 163),
  (0.039592778881985244, 166, 2, 146),
  (0.058459978795153446, 248, 2, 212),
  (0.04065778191104184, 127, 1, 109),
  (0.39543164993065977, 187, 0, 157),
  (0.012366140005544434, 174, 0, 152),
  (0.1280160306535343, 118, 1, 105),
  (0.044916596730138966, 181, 0, 151),
  (0.03567587642600943, 195, 1, 167),
  (0.016812285044496028, 116, 0, 116),
  (0.017364028339030654, 125, 0, 79),
  (0.18253351158857745, 148, 1, 129),
  (0.014923487129204905, 268, 0, 221),
  (0.08833750389838071, 125, 0, 104),
  (0.11430282142271841, 187, 1, 170),
  (0.046582615616139356, 154, 1, 140),
 

In [8]:
stats

{'better_after': 0, 'memory_hits': 182, 'worse_after': 152}